# data

In [1]:
import numpy as np
import pandas as pd

path = r"E:\Code\github"


mask_matrix = np.load(f"{path}\\mask_matrix.npy")
mask_data = np.load(f"{path}\\mask_data.npy")


print("mask_matrix shape:", mask_matrix.shape)
print("mask_data shape:", mask_data.shape)

total_elements = mask_data.size
missing_elements = np.isnan(mask_data).sum()
missing_ratio = missing_elements / total_elements

print(f"\nmasked_data Missing proportion: {missing_ratio:.2%} ({missing_elements} / {total_elements})")

mask_matrix shape: (300, 500, 6)
mask_data shape: (300, 500, 6)

masked_data Missing proportion: 2.00% (18000 / 900000)


In [3]:
import numpy as np

load_dir = r"E:\Code\github"

# 读取
LS1 = np.load(os.path.join(load_dir, "LS1.npy"))
LS2 = np.load(os.path.join(load_dir, "LS2.npy"))

print("LS1 shape:", LS1.shape)
print("LS2 shape:", LS2.shape)


LS1 shape: (300, 300)
LS2 shape: (500, 500)


In [4]:
# import sys
# sys.path.append(r"E:\Code\TDMIDR")
# Import our package
from tdmidr.model import TDMIDR
from tdmidr.trainer import train
# Import rclr
from rclr_utils.transforms import matrix_rclr, tensor_rclr, mask_value_only

from sklearn.model_selection import KFold
import collections
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

params = {
    'rank_1': 3,
    'rank_2': 4,
    'rank_3': 3,
    'lambda_var': 0.006536173305988504,
    'lambda_graph': 0.00013135506911867418,
    'lambda_smooth': 3.854275076893828e-05,
    'lambda_L1': 1.661774158540907e-05,
    'lambda_L2': 0.07914674388602203
}


dense_tensor = mask_data.copy()
dense_tensor_clr = tensor_rclr(np.nan_to_num(dense_tensor, nan=0))


train_data = mask_data.copy()

# rclr 
train_clr = tensor_rclr(np.nan_to_num(train_data, nan=0))


train_clr = torch.tensor(train_clr, dtype=torch.float32).to(device)
# mask_tensor = torch.from_numpy(dense_tensor_clr.copy()).float().to(device)
mask_tensor = torch.from_numpy(mask_data.copy()).float().to(device)

train_mask = torch.tensor(~np.isnan(mask_data), dtype=torch.bool).to(device)
test_mask = train_mask.clone()
LS1_tensor = torch.from_numpy(LS1.copy()).float().to(device)
LS2_tensor = torch.from_numpy(np.array(LS2).copy()).float().to(device)


M, N, T = train_data.shape

model = TDMIDR(input_shape=(M, N, T), ranks=(3, 4, 3), lambda_var=params['lambda_var'], lambda_smooth=params['lambda_smooth'], 
                     lambda_graph=params['lambda_graph'], lambda_L2=params['lambda_L2'],lambda_L1=params['lambda_L1'], p=2, 
                     device=device, X=train_clr, mask=train_mask, L1=LS1_tensor, L2=LS2_tensor,use_hosvd=True).to(device)


final_results = train(
    model, train_clr, mask_tensor, train_mask, test_mask, num_epochs=400, lr=1e-3)

print(final_results.keys())


Train  - MSE: 0.0014, MAE: 0.0033, RMSE: 0.0368
Val - MSE: 0.0014, MAE: 0.0033, RMSE: 0.0368
Train  - MSE: 0.0006, MAE: 0.0031, RMSE: 0.0252
Val - MSE: 0.0006, MAE: 0.0031, RMSE: 0.0252
Train  - MSE: 0.0003, MAE: 0.0028, RMSE: 0.0166
Val - MSE: 0.0003, MAE: 0.0028, RMSE: 0.0166
dict_keys(['X_hat', 'G_hat', 'U_hat', 'V_hat', 'W_hat', 'A_hat'])


In [5]:
# labels：
labels_df = pd.read_csv("E:/Code/github/SY.csv", index_col="SubjectID")

y = labels_df['group'].values  #
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], d

In [6]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, average_precision_score, adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.metrics import precision_recall_curve
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

our_result = final_results["U_hat"].cpu().detach().numpy() 
our_result


kf = KFold(n_splits=5, shuffle=True, random_state=42)

history_classification = {
    'auc': [],
    'aupr': [],
    'accuracy': [],
#     'recall': [],
#     'precision': [],
    'f1_score': []
}


for fold, (train_idx, test_idx) in enumerate(kf.split(our_result)):
    print(f"==  {fold+1}  ==")
   
    X_train, X_test = our_result[train_idx], our_result[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    
    mlp = MLPClassifier(hidden_layer_sizes=(48,), random_state=1234)
    mlp.fit(X_train, y_train)

    y_prob = mlp.predict_proba(X_test)[:, 1]  
    
        

    precision, recall, thresholds = precision_recall_curve(y_test, y_prob)


    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]

    y_pred = (y_prob >= best_threshold).astype(int)

    auc = roc_auc_score(y_test, y_prob)  # AUC (Area Under ROC Curve)
    aupr = average_precision_score(y_test, y_prob)  # AUPR (Area Under Precision-Recall Curve)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    

    history_classification['auc'].append(auc)
    history_classification['aupr'].append(aupr)
    history_classification['accuracy'].append(accuracy)
#     history_classification['recall'].append(recall)
#     history_classification['precision'].append(precision)
    history_classification['f1_score'].append(f1)
    

print(f"AUC: {np.mean(history_classification['auc'])}")
print(f"AUPR: {np.mean(history_classification['aupr'])}")
print(f"ACC: {np.mean(history_classification['accuracy'])}")
# print(f"Recall: {np.mean(history_classification['recall'])}")
# print(f"Precision: {np.mean(history_classification['precision'])}")
print(f"F1-score: {np.mean(history_classification['f1_score'])}")    

==  1  ==
==  2  ==
==  3  ==
==  4  ==
==  5  ==
AUC: 0.5719414225346743
AUPR: 0.5813270913045259
ACC: 0.5599999999999999
F1-score: 0.6865153851802893
